In [1]:
import pandas as pd
import re
import numpy as np
from fuzzywuzzy import fuzz

In [4]:
df_lobby_2018 = pd.read_csv('data/lobbyist(2017-2018).csv')
df_lobby_2016 = pd.read_csv('data/lobbyist(2015-2016).csv')
df_contribuion = pd.read_csv('data/contribution_processed.csv')

# 2018 Lobbyist

In [126]:
lobby_list= df_lobby_2016.append(df_lobby_2018)['Name'].unique()

In [68]:
lobby_list.shape

(3637,)

In [109]:
donor_list = df_contribuion['Donor_Name'].unique()

In [70]:
donor_list.shape

(51219,)

In [134]:
def clean_text(text):
    '''
    Remove special charecters, numbers
    Return lower case alphabet charecters only
    '''
    text = str(text).upper()
    text = re.sub('[^A-Za-z]+', '', text)
    return text

def takeoff_par(name):
    name = str(name)
    head, sep, tail = name.partition('(')
    return head

In [130]:
lobby_list = [takeoff_par(x) for x in lobby_list]
lobby_list_clean = [clean_text(x) for x in lobby_list]

In [131]:
donor_list_clean = [clean_text(x) for x in donor_list]

In [132]:
len(donor_list_clean)

51219

In [133]:
len(lobby_list_clean)

3637

In [142]:
match = []
donor_list_clean = np.array(donor_list_clean)
lobby_list_clean = np.array(lobby_list_clean)
for donor in donor_list_clean:
    for lobby in lobby_list_clean:
        if fuzz.ratio(donor, lobby) >= 85:
            match.append(donor)

In [144]:
match_un = np.array(match.copy())

In [149]:
match_set = set(match.copy())

In [152]:
len(match_set)

2657

In [153]:
df_contribuion['Donor_clean'] = df_contribuion['Donor_Name'].apply(clean_text)

In [155]:
def lobby_search(name):
    return name in match_set

In [156]:
df_contribuion['Lobbyist'] = df_contribuion['Donor_clean'].apply(lobby_search)

In [158]:
df_contribuion['Lobbyist'].value_counts()

False    101161
True      17300
Name: Lobbyist, dtype: int64

In [161]:
df_contribuion.drop(columns=['Donor_clean'], inplace=True)

In [163]:
df_contribuion

,Unnamed: 0,Address,Amount,City,Contributor_ID,Contributor_Type,Date,Employer,First_Name,Full_Name,...,Line_Sequence,Occupation,Principal_Officer,Report_ID,State,Zip,Donor_Name,Industry,PAC,Lobbyist
0,7373,270 S. Common Street,500.00,Lynn,NaN,Individual,2015-01-01,McGrath Enterprises,Patrick,Daniel F. Cahill,...,9166451,Developer,NaN,217037,MA,01905,PATRICK MCGRATH,NaN,NaN,False
1,7374,186 Perkins Row,250.00,Topsfield,NaN,Individual,2015-01-15,Self,Thomas,Daniel F. Cahill,...,9166452,Attorney,NaN,217037,MA,01983,THOMAS DEMAKIS,Healthcare,NaN,False
2,7634,11 Lake Ave.,25.00,Lynn,NaN,Individual,2015-03-02,NaN,Loretta,Daniel F. Cahill,...,9168914,NaN,NaN,217124,MA,10904,LORETTA O'DONNELL,NaN,NaN,False
3,7635,7 William St.,50.00,Lynn,NaN,Individual,2015-03-02,NaN,Stephen,Daniel F. Cahill,...,9168915,NaN,NaN,217124,MA,01904,STEPHEN SPENCER,NaN,NaN,False
4,7636,20 Bulfinch Ter.,50.00,Lynn,NaN,Individual,2015-03-02,NaN,Ann Marie,Daniel F. Cahill,...,9168916,NaN,NaN,217124,MA,01902,ANN MARIE LEONARD,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118456,513155,45 Winthrop Street,50.00,Boston,NaN,Individual,2018-09-06,NaN,Np,Chynah Tyler,...,13404233,NaN,NaN,712794,MA,02119,NP RIGGS,NaN,NaN,False
118457,513156,50 Forsyth Street,150.00,Boston,NaN,Individual,2018-09-06,Self,Chicken,Chynah Tyler,...,13404234,Resturant,NaN,712794,MA,02115,CHICKEN LOUS,Self-Employed,NaN,False
118458,513157,Actblue,1413.42,Actblue,NaN,Individual,2018-08-28,Actblue,Actblue,Chynah Tyler,...,13404235,actable,NaN,712794,MA,NaN,ACTBLUE ACTBLUE,NaN,NaN,False
118459,513158,60 Winthrop St,50.00,Roxbury,NaN,Individual,2018-09-06,NaN,Carl,Chynah Tyler,...,13404236,NaN,NaN,712794,MA,02119,CARL TODISCO,NaN,NaN,False


In [166]:
df_contribuion.to_csv('data/contribution_processed.csv')